In [4]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt

fatal: destination path 'StockSellRulesChecker' already exists and is not an empty directory.
     |████████████████████████████████| 9.5MB 5.3MB/s 
     |████████████████████████████████| 1.3MB 47.0MB/s 
     |████████████████████████████████| 143kB 54.1MB/s 
     |████████████████████████████████| 296kB 51.3MB/s 
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=15193 sha256=d8fb2baeef55a62ecac1fbc7f103f2c9fd03265a4f274e6df6eb15fdae7dc824
  Stored in directory: /root/.cache/pip/wheels/6d/fa/7c/4cc075d8af3f8ed083dd112b1a2b09888b9a9a2964c907291f
  Created wheel for stockquotes: filename=stockquotes-2.0.1-cp37-none-any.whl size=4472 sha256=dfff878ac361b6cd05f75c7c9355110163ec6fc9f1b4b83c0b1047b127c3a50d
  Stored in directory: /root/.cache/pip/wheels/b6/5a/fa/c250a28690344108071fb9b46761d5ac3bb23798260f8ecff1
  Created wheel for yfinance: filename=yfinance-0.1.55-py2.py3-none-any.whl size=22616 sha256=53486cf77117b320d959eee6b888d80ec1afc6cc88f0f794e7e5a6c89617937f

In [5]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

In [7]:
filters = ['fa_eps5years_o20', 'fa_epsqoq_o20',
           'fa_epsyoy_o20','fa_sales5years_o20','fa_salesqoq_o20',
           'sh_curvol_o200'] 
stock_list = Screener(filters=filters, table='Performance', order='price')  

In [8]:
print(stock_list)

No. | Ticker | Perf Week | Perf Month | Perf Quart | Perf Half | Perf Year | Perf YTD | Volatility W | Volatility M | Recom | Avg Volume | Rel Volume | Price   | Change | Volume    
--- | ------ | --------- | ---------- | ---------- | --------- | --------- | -------- | ------------ | ------------ | ----- | ---------- | ---------- | ------- | ------ | ----------
1   | VTVT   | 17.00%    | 7.04%      | -6.77%     | 52.11%    | 57.49%    | 55.38%   | 7.84%        | 10.69%       | 2.00  | 3.49M      | 0.65       | 2.89    | 4.33%  | 2,274,688 
2   | OPK    | 2.65%     | -17.29%    | 4.27%      | 46.37%    | 172.94%   | 17.47%   | 5.90%        | 6.91%        | 1.80  | 8.40M      | 0.59       | 4.64    | -1.69% | 4,981,445 
3   | SIGA   | -7.55%    | 3.12%      | -5.03%     | -6.51%    | 65.66%    | -9.08%   | 4.55%        | 3.98%        | -     | 240.29K    | 1.10       | 6.61    | -0.90% | 265,285   
4   | CTMX   | 3.41%     | -5.67%     | 12.01%     | 18.25%    | 135.18%   | 29.62%   | 4.

In [9]:
def moving_average(yahoo_df, days):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=days)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())


screened_stocks = {}
for stock in tqdm.tqdm(stock_list):
    screened_stocks[stock['Ticker']] = {}
    
    finviz_stats = finviz.get_stock(stock['Ticker'])
    try:
        yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
    except:
        continue
    SMA200_value = moving_average(yahoo_df, days=200)
    SMA150_value = moving_average(yahoo_df, days=150)
    SMA50_value = moving_average(yahoo_df, days=50)
    SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))
    SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))
    prev_close = float(finviz_stats['Prev Close'].replace("$",""))
    week52_high, week52_low = week52_low_high(yahoo_df)
    
    screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
    screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
    screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
    screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
    screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
    screened_stocks[stock['Ticker']]['prev_close'] = prev_close
    screened_stocks[stock['Ticker']]['week52_high'] = week52_high
    screened_stocks[stock['Ticker']]['week52_low'] = week52_low
    
    # Positive 200d MA positive
    if SMA200_percent > 0:
        SMA200_slope_rule = True
    else:
        SMA200_slope_rule = False
    screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
        
    # 150d MA greater than 200d MA
    if SMA150_value > SMA200_value:
        SMA150_greater_SMA200_rule = True
    else:
        SMA150_greater_SMA200_rule = False
    screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
        
    # 50d MA greater than 150d MA
    if SMA50_value > SMA150_value:
        SMA50_greater_SMA150_rule = True
    else:
        SMA50_greater_SMA150_rule = False
    screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
        
    # Close above 50d MA
    if prev_close > SMA50_value:
        close_greater_SMA50_rule = True
    else:
        close_greater_SMA50_rule = False
    screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
        
    # 52 week high low span rule
    if 0.75*week52_high > 1.25*week52_low:
        week52_span_rule = True
    else:
        week52_span_rule = False
    screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
    
    # Close above 52 week high - 25%
    if prev_close > 0.75*week52_high:
        close_above_52weekhigh_rule = True
    else:
        close_above_52weekhigh_rule = False
    screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule

output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

    


100%|██████████| 45/45 [00:31<00:00,  1.41it/s]


In [12]:
df_out

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,prev_close,week52_high,week52_low,SMA200_slope_rule,SMA150_greater_SMA200_rule,SMA50_greater_SMA150_rule,close_greater_SMA50_rule,week52_span_rule,close_above_52weekhigh_rule
0,VTVT,2.085255,2.166176,2.506389,30.86,21.78,2.77,3.190000,1.490000,True,True,True,True,True,True
1,OPK,4.247810,4.488039,4.984444,11.35,-3.58,4.72,5.990000,1.150000,True,True,True,False,True,True
2,SIGA,6.940584,6.946765,6.599722,-2.49,-3.15,6.67,7.760000,3.990000,False,True,False,True,True,True
3,CTMX,7.376971,7.547255,8.074444,12.63,8.12,8.76,14.640000,3.610000,True,True,True,True,True,False
4,RBBN,6.169489,6.900588,8.820555,54.52,4.12,8.65,11.140000,2.030000,True,True,True,False,True,True
5,DVAX,5.836496,6.159020,8.984444,53.56,27.45,10.21,11.700000,2.270000,True,True,True,True,True,True
6,WKHS,23.612810,23.518775,26.780139,-16.99,-36.03,17.83,41.340000,1.470000,False,False,True,False,True,False
7,PLYM,14.039197,14.447353,15.575000,23.13,11.39,17.35,17.350000,7.900000,True,True,True,True,True,True
8,VALE,14.509343,15.684608,17.364167,30.33,-1.33,17.24,18.940001,6.580000,True,True,True,False,True,True
9,GTN,16.576569,17.375392,18.797222,25.75,9.23,20.17,20.660000,8.810000,True,True,True,True,True,True


In [ ]:
df_out.to_csv("screened_stocks.csv")